In [1]:
"""
This is a library of functions for working 
with fastq files and iBest software.
"""
from __future__ import print_function
import sys, os, subprocess
import string
import numpy as np
import random
import pandas as pd
from Bio import pairwise2
import fnmatch
import filecmp


# find all files, output names
def find_fastq_files(directory, pattern):
    """Walks the directory structure, appending filenames to an array"""
    filenames = []
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                filenames.append(filename)
    filenames.sort()
    return filenames


# basic call to the shell application we are testing
def sub_process(command):
    return subprocess.check_output(
        command, stderr=subprocess.STDOUT, shell=True)

def file_compare(command, expected, returned):
    # testing expected file output
    subprocess.check_output(command, stderr=subprocess.STDOUT, shell=True)
    return filecmp.cmp(expected, returned)


def parse_fastq(filename):
    # output the file to a dictionary
    with open(filename) as f:
        lines = f.readlines()
    head = [item[:-1] for item in lines[0::4]]
    read = [item[:-1] for item in lines[1::4]]
    qual = [item[:-1] for item in lines[3::4]]
    return {'Header': head, 'Sequence': read, 'QScore': qual}

def word_count(wordstring):
    wordlist = list(wordstring)

    wordfreq = []
    for w in wordlist:
        wordfreq.append(wordlist.count(w))

    print ("String" + "\n" + wordstring +"\n")
    print ("List" + "\n" + str(wordlist) + "\n")
    print ("Frequencies" + "\n" + str(wordfreq) + "\n")
    print ("Pairs" + "\n" + str(zip(wordlist, wordfreq)))
    
def gen_sequence(length):
    """Return a randomly generated nucleotied sequence"""
    return ''.join(random.choice('ACTG') for _ in range(length))

def non_sequence(length):
    """Returns a randomly generated sequence with N's"""
    return ''.join(random.choice('ACTGNNN') for _ in range(length))

def constrained_sum_sample_pos(n, total):
    """Return a randomly chosen list of n positive integers summing to total.
    Each such list is equally likely to occur."""

    dividers = sorted(random.sample(xrange(1, total), n - 1))
    return [a - b for a, b in zip(dividers + [total], [0] + dividers)]

def gen_qscore(length, score):
    """Return a fake qscore of length length"""
    myScore = constrained_sum_sample_pos(length,score)
    return ''.join([chr(i+33) for i in myScore])

def gen_ID(alpha):
    """Return an ID for the fastq file five characters long using the
    supplied alphabet"""
    return ''.join(random.choice(alpha) for _ in range(5))

def NoN(s):
    """Return the longest string not containing N's"""
    s1 = "NGNTAGAAAAAAAAAAAAAAAAAAAAAAAGTAAAAACAAAGN"
    s2 = "GAGAAAAAAAAAAAAAAAAAAAAAAAGTAAAAACAAAGNNNNNNNNNNNNNNN"
    s3 = "GAGAAAAAAAAAAAAAAAAAAAAAAAGTAAAAACAAAG"
    s4 = "NNNNNGAGAAAAAAAAAAAAAAAAAAAAAAAGTAAAAACAAAG"
    s5 = "NNNNNNNNNNNNNNNNNNNNNNNNNNANNNNNNNNNNNNNNNN"
    
    s=[s1,s2,s3,s4,s5]
    
    max_end = 0
    max_start = 0;
    current_end = 0;
    current_start = 0;

    for c in range(0, len(s)):
        if (s[c] == 'N'):
            if c - current_start + 1 > max_end - max_start:
                max_end = c;
                max_start = current_start;
                current_start = c + 1;
        
    if c - current_start > max_end - max_start:
        max_end = c + 1
        max_start = current_start
    return s[max_start:max_end]

def return_binary(st):
    # converts string to binary
     return ' '.join(format(x, 'b') for x in bytearray(st))
        
def return_reverse_complement(seq):
    # returns the reverse complement of a sequence
    seq1 = 'ATCGTAGCatcgtagc'
    seq_dict = {seq1[i]: seq1[i + 4]
                for i in range(16) if i < 4 or 8 <= i < 12}
    return "".join([seq_dict[base] for base in reversed(seq)])

def gen_fake_fastq(readLength, qScore):  
    """Returns fake randomly generated fastq data
    as a dictionary"""
    alphabet = "'AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz0123456789'"
    store={}
    id_Strings=[]
    seq_R1=[]
    seq_R2=[]
    q_R1=[]
    q_R2=[]
    
    id_Strings.append("@"+gen_ID(alphabet))
    seq_R1.append(gen_sequence(readLength))
    seq_R2.append(gen_sequence(readLength))
    q_R1.append(gen_qscore(readLength,qScore))
    q_R2.append(gen_qscore(readLength,qScore))
        
    store['ID']=id_Strings
    store['Seq_R1']=seq_R1
    store['Seq_R2']=seq_R2
    store['Q_R1']=q_R1
    store['Q_R2']=q_R2
    
    return pd.DataFrame(store)

def trim_adaptor(seq, adaptor, num_errors, right_side=True):
    """
    The Biopython library contains a Python local alignment function suitable
    for quick alignment of short regions. We use this to align an adaptor 
    region to a sequence and calculate the number of differences in the 
    aligned region. To handle a common case where we can find the exact
    adaptor sequence, we first do an string match. 
    This avoids the alignment overhead in many cases:
    """

    gap_char = '-'
    exact_pos = str(seq).find(adaptor)
    if exact_pos >= 0:
        seq_region = str(seq[exact_pos:exact_pos+len(adaptor)])
        adapt_region = adaptor
    else:
        seq_a, adaptor_a, score, start, end = pairwise2.align.localms(str(seq),
                str(adaptor), 5.0, -4.0, -9.0, -0.5,
                one_alignment_only=True, gap_char=gap_char)[0]
        adapt_region = adaptor_a[start:end]
        seq_region = seq_a[start:end]
    matches = sum((1 if s == adapt_region[i] else 0) for i, s in
            enumerate(seq_region))
    # too many errors -- no trimming
    if (len(adaptor) - matches) > num_errors:
        return seq
    # remove the adaptor sequence and return the result
    else:
        return _remove_adaptor(seq, seq_region.replace(gap_char, ""),
                right_side)

def _remove_adaptor(seq, region, right_side=True):
    """
    Remove an adaptor region and all sequence to the right or left.
    If the alignment contains fewer than the specified number of
    errors we’ve found an adaptor and remove it, returning the trimmed
    sequence. The attribute right_side allows us to specify whether
    the trimming should be done on the right (3′ end) or the left (5′ end):
    """
    if right_side:
        try:
            pos = seq.find(region)
        # handle Biopython SeqRecords
        except AttributeError:
            pos = seq.seq.find(region)
        return seq[:pos]
    else:
        try:
            pos = seq.rfind(region)
        # handle Biopython SeqRecords
        except AttributeError:
            pos = seq.seq.rfind(region)
        return seq[pos+len(region):]

In [ ]:
"""
Here is an example script using this function along with Biopython
to trim reads from a FASTQ format file. Each read is analyzed to
determine if it contains the adaptor, with up to 2 errors.
If the adaptor is identified and trimmed, the final useful
read is written to a FASTA format output file.
"""

from __future__ import with_statement
import sys
import os
 
from Bio import SeqIO
# The in_file var will need to be changed
in_file = "/Users/ChrisM/Documents/workspace/iBest/fastaFiles/rawData/NTC_R1.fastq"
adaptor_seq = "ACACTCTTTCCCTACACGACGCTCTTCCGATCT"
reverse_complement = return_reverse_complement(adaptor_seq)
num_errors = 2
base, ext = os.path.splitext(in_file)
out_file = "%s-trimmed.fa" % (base)

with open(in_file) as in_handle:
    with open(out_file, "w") as out_handle:
        for rec in SeqIO.parse(in_handle, "fastq"):
            trim = trim_adaptor(rec.seq, adaptor_seq, num_errors)
            if len(trim) > 0 and len(trim) < len(rec):
                rec.letter_annotations = {}
                rec.seq = trim
                SeqIO.write([rec], out_handle, "fasta")